# 1. 라이브러리 및 파일 불러오기

## 1.1 라이브러리 불러오기

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install sentence-transformers

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import re
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=b8aaa11f275e1064662a94f98751919f04d98e1caef675cfa78ba89e05d785c7
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342be

/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(


In [ ]:
# 전처리 완료한 데이터 파일명 입력
train = pd.read_csv("../train_all[0202].csv")
train.info()

In [ ]:
train.tail(10)

## 1.3 데이터 구조 확인

In [ ]:
# 개체명 엔티티 타입(태그) 확인
train['tar'].unique()

In [ ]:
# 불필요한 인덱스 컬럼 제거
train.drop('Unnamed: 0', axis = 1, inplace=True)
train[:20]

# 2. 데이터 수정

## 2.1 리스트 형식으로 변환

In [ ]:
data = [list(x) for x in train[['index', 'src', 'tar']].to_numpy()]

## 2.2 태그를 딕셔너리 형태로 변환

In [ ]:
label = train['tar'].unique().tolist()
label_dict = {word:i for i, word in enumerate(label)}
label_dict.update({"[PAD]":len(label_dict)})
index_to_ner = {i:j for j, i in label_dict.items()}

In [ ]:
print(label_dict)

{'-': 0, 'OI': 1, 'NM': 2, 'PN': 3, 'UI': 4, 'BI': 5, 'AI': 6, nan: 7, '[PAD]': 8}


In [ ]:
print(index_to_ner)

{0: '-', 1: 'OI', 2: 'NM', 3: 'PN', 4: 'UI', 5: 'BI', 6: 'AI', 7: nan, 8: '[PAD]'}


## 2.3 단어와 태그를 한묶음으로 생성

In [ ]:
tups = []
temp_tup = []
temp_tup.append(data[0][1:])
sentences = []
targets = []
for i, j, k in data:
  
    if i != 1:
        temp_tup.append([j,label_dict[k]])
    if i == 1:
        if len(temp_tup) != 0:
            tups.append(temp_tup)
            temp_tup = []
            temp_tup.append([j,label_dict[k]])

tups.pop(0)

## 2.4 단어와 태그 개별 리스트로 저장

In [ ]:
sentences = []
targets = []
for tup in tups:
    sentence = []
    target = []
    sentence.append("[CLS]")
    target.append(label_dict['-'])
    for i, j in tup:
        sentence.append(i)
        target.append(j)
    sentence.append("[SEP]")
    target.append(label_dict['-'])
    sentences.append(sentence)
    targets.append(target)

In [ ]:
targets[0]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

# 3. 버트 인풋 만들기

## 3.1 버트 토크나이저 다운로드

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer_config.json


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size":

In [ ]:
print(len(tokenizer))

119547


## 3.2 버트 토크나이저 사전에 지명 추가

In [ ]:
'''#추가할 지명 사전을 리스트 형태로 변환
with open('주소 vocab.txt', 'r', encoding = 'cp949') as f:
    list_file = []
    for line in f:
        list_file.append(line)'''

"#추가할 지명 사전을 리스트 형태로 변환\nwith open('주소 vocab.txt', 'r', encoding = 'cp949') as f:\n    list_file = []\n    for line in f:\n        list_file.append(line)"

In [ ]:
'''# 줄바꿈 표시 제거
search = '\n'
for i, word in enumerate(list_file):
    if search in word: 
        list_file[i] = word.strip(search)'''

"# 줄바꿈 표시 제거\nsearch = '\n'\nfor i, word in enumerate(list_file):\n    if search in word: \n        list_file[i] = word.strip(search)"

In [ ]:
'''# 주소를 vocab에 추가
tokenizer.add_tokens(list_file)'''

Streaming output truncated to the last 5000 lines.
Adding 합내로593번길 to the vocabulary
Adding 합내로796번길 to the vocabulary
Adding 합내로836번길 to the vocabulary
Adding 합내로854번길 to the vocabulary
Adding 합내로933번길 to the vocabulary
Adding 합덕교동1길 to the vocabulary
Adding 합덕교동2길 to the vocabulary
Adding 합덕대덕로 to the vocabulary
Adding 합덕대호로 to the vocabulary
Adding 합덕도곡길 to the vocabulary
Adding 합덕리 to the vocabulary
Adding 합덕산단1로 to the vocabulary
Adding 합덕산단2로 to the vocabulary
Adding 합덕산단3로 to the vocabulary
Adding 합덕산단4로 to the vocabulary
Adding 합덕성당1길 to the vocabulary
Adding 합덕성당2길 to the vocabulary
Adding 합덕시장로 to the vocabulary
Adding 합덕읍 to the vocabulary
Adding 합동 to the vocabulary
Adding 합리 to the vocabulary
Adding 합리길 to the vocabulary
Adding 합목1길 to the vocabulary
Adding 합목1안길 to the vocabulary
Adding 합목2길 to the vocabulary
Adding 합목3길 to the vocabulary
Adding 합목리 to the vocabulary
Adding 합사길 to the vocabulary
Adding 합사길138번길 to the vocabulary
Adding 합사길315번길 to the vocabulary
Adding 합사

116400

In [ ]:
'''modelTokenizer = BertModel.from_pretrained("bert-base-cased")
modelTokenizer.resize_token_embeddings(len(tokenizer)) 
# The new vector is added at the end of the embedding matrix'''

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/pytorch_model.bin
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClass

Embedding(235947, 768)

In [ ]:
'''current_size = tokenizer.vocab_size
tokenizer.vocab_size, current_size, len(tokenizer.vocab)'''

(119547, 119547, 119547)

In [ ]:
'''#vocab_size도 사전 안에 토큰 개수가 추가된 만큼 증가한 것을 확인할 수 있음
modelTokenizer.vocab_size, current_size, len(tokenizer.vocab)'''

(235947, 119547, 119547)

In [ ]:
# vocab 안에 포함되지 않은 '흰색돌로'는 최소한의 단위로 쪼개졌지만, vocab에 추가된 주소 '흰돌로'는 더 쪼개지지 않음.
tokenizer.tokenize("흰돌로를 찾고 있는 것인지, 아니면 흰색돌로를 찾고 있는 것인지 알려주시기 바랍니다.")

['흰돌로',
 '를',
 '찾',
 '##고',
 '있는',
 '것',
 '##인',
 '##지',
 ',',
 '아',
 '##니',
 '##면',
 '흰',
 '##색',
 '##돌',
 '##로',
 '##를',
 '찾',
 '##고',
 '있는',
 '것',
 '##인',
 '##지',
 '알',
 '##려',
 '##주시',
 '##기',
 '바',
 '##랍',
 '##니다',
 '.']

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        word = str(word)
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        tokenized_sentence.extend(tokenized_word)
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
tokenized_texts_and_labels = [
                              tokenize_and_preserve_labels(sent, labs)
                              for sent, labs in zip(sentences, targets)]
# [(문장, 개체들), (문장, 개체들),...] 형식으로 저장됨.

In [ ]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
labels[1]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 0]

In [ ]:
# max_len 변경 가능
print(np.quantile(np.array([len(x) for x in tokenized_texts]), 0.99))
max_len = 500
bs = 32

1271.0


#### 버트 인풋
- input_ids : 문장이 토크나이즈 된 것이 숫자로 바뀐 것
- attention_masks : 문장이 토크나이즈 된 것 중에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 마스킹
- [input_ids, attention_masks]가 인풋으로 들어감

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype = "int", value=tokenizer.convert_tokens_to_ids("[PAD]"), truncating="post", padding="post")

In [ ]:
input_ids[1]

array([   101,   9665, 110817,   9590,  12424,  10013,  14423, 119401,
        10622,   8896, 119235,  11261,   9691,  85634,  51076,   8857,
        43022,  10622,   9956,   9539,  16605,  12310,  18784,  10892,
         9706, 118718,  41850,   9930,  10013,  14423, 119401, 118611,
        11467,    129,  21789,  38851,  10739,   8987,  16985,  70067,
         9956,  62849,  15303,   8896, 119235,  59894,   9965,  41850,
         8924,  31531,  77039,   9069,  40032,   9751,  97371,   9956,
         9559,  12605,  88332,  10739, 119354,  80174,  20308,  23635,
          124,  10954,   8887,  47289,  10739,  37909,   9531,  11467,
          129,  21789,  38851,  18784,  16709,  10929,  14279, 119108,
         9074,   8988,  73969,   9960,  21711,  16139,  11664,   9956,
         8887,  47289,  13764,   9670,  75965,  14871,    130,    120,
          128,    102,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [ ]:
tags = pad_sequences([lab for lab in labels], maxlen=max_len, value=label_dict["[PAD]"], padding='post',\
                     dtype='int', truncating='post')

In [ ]:
tags[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,

In [ ]:
attention_masks = np.array([[int(i != tokenizer.convert_tokens_to_ids("[PAD]")) for i in ii] for ii in input_ids])

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.2)

In [ ]:
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.2)

### TPU 작동을 위해 실행

In [ ]:
# # TPU 작동을 위해 실행
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
SEQ_LEN = max_len
def create_model():
  model = TFBertModel.from_pretrained("bert-base-multilingual-cased", from_pt=True, num_labels=len(label_dict), output_attentions = False,
    output_hidden_states = False)

  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids') # 토큰 인풋
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks') # 마스크 인풋

  bert_outputs = model([token_inputs, mask_inputs])
  bert_outputs = bert_outputs[0] # shape : (Batch_size, max_len, 30(개체의 총 개수))
  nr = tf.keras.layers.Dense(30, activation='softmax')(bert_outputs) # shape : (Batch_size, max_len, 30)
  
  nr_model = tf.keras.Model([token_inputs, mask_inputs], nr)
  
  nr_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00002), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
  nr_model.summary()
  return nr_model

# 4. 훈련 및 성능 검증

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
# TPU를 활용하기 위해 context로 묶어주기
with strategy.scope():
  nr_model = create_model()
  nr_model.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=10, shuffle=False, batch_size=bs)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_lay

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 500)]        0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 500)]        0           []                               
                                                                                                  
 tf_bert_model_2 (TFBertModel)  TFBaseModelOutputWi  177853440   ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_masks[0][0]']            
                                tentions(last_hidde                                               
                                n_state=(None, 500,                                         

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10


954/954 [==============================] - 283s 209ms/step - loss: 0.0225 - sparse_categorical_accuracy: 0.9947 - val_loss: 0.0048 - val_sparse_categorical_accuracy: 0.9987
Epoch 2/10
954/954 [==============================] - 184s 193ms/step - loss: 0.0047 - sparse_categorical_accuracy: 0.9987 - val_loss: 0.0043 - val_sparse_categorical_accuracy: 0.9989
Epoch 3/10
954/954 [==============================] - 185s 194ms/step - loss: 0.0037 - sparse_categorical_accuracy: 0.9990 - val_loss: 0.0043 - val_sparse_categorical_accuracy: 0.9989
Epoch 4/10
954/954 [==============================] - 185s 194ms/step - loss: 0.0030 - sparse_categorical_accuracy: 0.9992 - val_loss: 0.0043 - val_sparse_categorical_accuracy: 0.9989
Epoch 5/10
954/954 [==============================] - 185s 194ms/step - loss: 0.0025 - sparse_categorical_accuracy: 0.9993 - val_loss: 0.0048 - val_sparse_categorical_accuracy: 0.9989
Epoch 6/10
954/954 [==============================] - 185s 194ms/step - loss: 0.0023 - spar

In [ ]:
'''# 만약 TPU를 사용하지 않고 GPU를 사용한다면
nr_model = create_model()
# Epoch 변경 가능
nr_model.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=5
             , shuffle=False, batch_size=bs)'''

## 4.1 결과 확인

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
y_predicted = nr_model.predict([val_inputs, val_masks])

In [ ]:
f_label = [i for i, j in label_dict.items()]
val_tags_l = [index_to_ner[x] for x in np.ravel(val_tags).astype(int).tolist()]
y_predicted_l = [index_to_ner[x] for x in np.ravel(np.argmax(y_predicted, axis=2)).astype(int).tolist()]
f_label.remove("[PAD]")

## 4.2 1차 TEST 시작

In [ ]:
print(classification_report(val_tags_l, y_predicted_l, labels=f_label))

# 5. 사전 추가

In [ ]:
print(list_file[0:100])

In [ ]:
print(y_predicted_l[:10])

In [ ]:
for i in val_inputs:
    for j in i:
        if j in list_file:
            y_predicted_l[i * 50 + j] = 'OI'

# 6. 샘플 데이터 테스트

## 6.1 한 문장씩 분류 결과 확인

In [ ]:
def ner_inference(test_sentence):
  

    tokenized_sentence = np.array([tokenizer.encode(test_sentence, max_length=max_len, truncation=True, padding='max_length')])
    tokenized_mask = np.array([[int(x!=1) for x in tokenized_sentence[0].tolist()]])
    ans = nr_model.predict([tokenized_sentence, tokenized_mask])
    ans = np.argmax(ans, axis=2)

    tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
    new_tokens, new_labels = [], []
    for token, label_idx in zip(tokens, ans[0]):
        if (token.startswith("##")):
            new_labels.append(index_to_ner[label_idx])
            new_tokens.append(token[2:])
        elif (token=='[CLS]'):
            pass
        elif (token=='[SEP]'):
            pass
        elif (token=='[PAD]'):
            pass
        elif (token != '[CLS]' or token != '[SEP]'):
            new_tokens.append(token)
            new_labels.append(index_to_ner[label_idx])

    for token, label in zip(new_tokens, new_labels):
        print("{}\t{}".format(label, token))

In [ ]:
ner_inference("충북 음성군 맹동면 용두로 544")

##  6.2 엑셀파일 분류 결과 확인

In [ ]:
import pandas as pd
from openpyxl import load_workbook
import csv

In [ ]:
def list_chunk(lst,n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

In [ ]:
def ner_to_excel(test_dir,filename):
    
    read_xlsx = load_workbook(test_dir)    
    read_sheet = read_xlsx.active
    name_col = read_sheet['A']
    names = []
    
    for cell in name_col:
        names.append(cell.value)
    
    result = ' '.join(s for s in names)
    sent = names
    new_tokens, new_labels = [], []
    
    for i in tqdm(range(len(sent))):
        
        tokenized_sentence = np.array([tokenizer.encode(sent[i], max_length=max_len, truncation=True, padding='max_length')])
        tokenized_mask = np.array([[int(x!=1) for x in tokenized_sentence[0].tolist()]])
        ans = nr_model.predict([tokenized_sentence, tokenized_mask])
        ans = np.argmax(ans, axis=2)

        tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
        
        for token, label_idx in zip(tokens, ans[0]):
            if (token.startswith("##")):
                new_labels.append(index_to_ner[label_idx])
                new_tokens.append(token[2:])
            elif (token=='[CLS]'):
                pass
            elif (token=='[SEP]'):
                pass
            elif (token=='[PAD]'):
                pass
            elif (token != '[CLS]' or token != '[SEP]'):
                new_tokens.append(token)
                new_labels.append(index_to_ner[label_idx])
        
    df = pd.DataFrame()
    df['token'] = new_tokens
    df['label'] = new_labels
    df.info()
    
#     filename = datetime.datetime.now().strftime("%m%d")+str(name)+str("_ner_result")
    df.to_excel(filename +'.xlsx', index = False)


In [ ]:
# ner_to_excel(ner분석할 엑셀파일, 파일에서 문장 구분할 길이, 저장할 파일명)
ner_to_excel(r'../샘플 데이터[0201].xlsx', '샘플 데이터 test_result')

1/1 [==============================] - 1s 594ms/step


1/1 [==============================] - 0s 391ms/step


1/1 [==============================] - 1s 560ms/step


1/1 [==============================] - 0s 469ms/step
